In [13]:
from scipy.optimize import minimize
import numpy.linalg as la
import numpy as np


### Penalty

\begin{align*}
\min_{\mathbf x}\quad&2x^2 + y^2 + z^2 - 20x -16y\\
\text{s.t}\quad&x^2 + y^2 + z^3 \le 1\\ 
&x+y-z=0
\end{align*}

In [14]:
x_sol = [0.507251382811985, 0.300469991645678, 0.807721374457662]

**Scipy library**

In [15]:
# Define la función objetivo
objective = lambda x: 2 * x[0]**2 + x[1]**2 + x[2]**2 - 20 * x[0] - 16 * x[1]

# Define las restricciones del problema
constraints = [
    {'type': 'ineq', 'fun': lambda x: 1 - x[0]**2 - x[1]**2 - x[2]**2},
    {'type': 'eq', 'fun': lambda x: x[0] + x[1] - x[2]}
]

# Resuelve el problema de optimización
res = minimize(objective, np.zeros(3), constraints=constraints)
res.x


array([0.50725139, 0.30046998, 0.80772137])

**Algorithm**

In [16]:
def external_penalty(f, g, h, x, eps=1e-5, MAX_ITER=100):
    print(f"{0:02d}, {x}")  # x[0]
    phi = lambda y: max(0, y)**2
    psi = lambda y: abs(y)**2
    alpha = lambda x: phi(g(x)) + psi(h(x))
    sub_problem = lambda x, u: f(x) + u * alpha(x)

    u = 5
    beta = 2

    for k in range(1, MAX_ITER):
        x = minimize(sub_problem, x, u).x
        error = u * alpha(x)
        print(f"{k:02d}, {x}, {error:0.8f}")
        if error < eps:
            break
        else:
            u = beta * u


In [17]:
f = lambda x: 2 * x[0]**2 + x[1]**2 + x[2]**2 - 20 * x[0] - 16 * x[1]
g = lambda x: x[0]**2 + x[1]**2 + x[2]**2 - 1
h = lambda x: x[0] + x[1] - x[2]
x = np.zeros(3)
external_penalty(f, g, h, x)


00, [0. 0. 0.]
01, [0.8271359  0.63769604 0.6544557 ], 4.63098804
02, [0.70137073 0.50160154 0.72904548], 3.00249382
03, [0.61728798 0.41287495 0.76980355], 1.77108055
04, [0.56653121 0.36045002 0.78966208], 0.97596231
05, [0.53814712 0.33155415 0.79902725], 0.51495343
06, [0.52304363 0.31630924 0.80347651], 0.26492456
07, [0.51523798 0.30846726 0.80562717], 0.13442600
08, [0.51126793 0.30448843 0.80668168], 0.06771763
09, [0.50926563 0.30248412 0.80720341], 0.03398673
10, [0.50826018 0.30147809 0.80746283], 0.01702553
11, [0.50775655 0.30097389 0.80759215], 0.00852080
12, [0.50750479 0.30072114 0.80765665], 0.00426239
13, [0.50737962 0.30059379 0.80768874], 0.00213156
14, [0.5073187  0.30052811 0.80770446], 0.00106585
15, [0.50729055 0.30049251 0.8077119 ], 0.00053291
16, [0.50728079 0.30046955 0.80771477], 0.00026582
17, [0.50728386 0.30044871 0.80771475], 0.00013338
18, [0.50731453 0.30040366 0.80770928], 0.00006636
19, [0.50738309 0.30031789 0.80769663], 0.00003155
20, [0.50738161 

### Barrier

\begin{align*}
\min_{\mathbf x}\quad&2x^2+y^2\\
\text{s.t}\quad&4x^2+y^2-2 \le 0\\ 
&4x+y+1\le0
\end{align*}

In [18]:
f = lambda x: 2 * x[0]**2 + x[1]**2
g = [lambda x: 4 * x[0]**2 + x[1]**2 - 2,
     lambda x: 4 * x[0] + x[1] + 1]
x_sol = [-2 / 9, -1 / 9]


**Scipy library**

In [19]:
# Define la función objetivo
# Define las restricciones del problema
constraints = [{'type': 'ineq', 'fun': lambda x: -gi(x)} for gi in g]

# Resuelve el problema de optimización
res = minimize(f, np.zeros(2), constraints=constraints)
res.x


array([-0.22222222, -0.11111111])

**Scipy library**

\begin{align*}
\min_{\mathbf x}\quad&{(x_1 - 2)}^4 + {(x_1 - 2x_2)}^2\\
\text{s.t}\quad&x_1^2-x_2 \le 0
\end{align*}

In [25]:
f = lambda x: (x[0] - 2)**4 + (x[0] - 2 * x[1])**2
g = [lambda x: -(x[0]**2 - x[1])]


In [26]:
# Define la función objetivo
# Define las restricciones del problema
constraints = [{'type': 'ineq', 'fun': lambda x: -gi(x)} for gi in g]

# Resuelve el problema de optimización
res = minimize(f, np.zeros(2), constraints=constraints)
res.x


array([1.97486175, 0.98741104])

In [29]:
def barrier_method(f, g, phi, x, u=10, beta=0.1, eps=1e-8, MAX_ITER=100):
    B = lambda x: sum([phi(gi(x)) for gi in g])
    sub_problem = lambda x, u: f(x) + u * B(x)
    for k in range(1, MAX_ITER):
        x = minimize(sub_problem, x, u).x
        error = u * B(x)
        print(f"{k:02d}, {x}, {error:0.8f}")
        if abs(error) < eps:
            break
        else:
            u = beta * u


In [40]:
x = np.array([0, 1.0])
u = 2
beta = 0.1 
phi_1 = lambda y: -1 / y
phi_2 = lambda y: np.log(min(1, -y))
# phi_3 = lambda y: -np.log(-y)
barrier_method(f, g, phi_1, x, u, beta)


01, [2.45645457 1.21745173], 0.41522055
02, [2.23615197 1.11641971], 0.05149389
03, [2.11680435 1.05818881], 0.00584339
04, [2.05613485 1.028043  ], 0.00062507
05, [2.02696278 1.01347877], 0.00006462
06, [2.01607319 1.00803632], 0.00000654
07, [2.01021679 1.00510836], 0.00000066
08, [2.01021679 1.00510836], 0.00000007
09, [2.01021679 1.00510836], 0.00000001


In [24]:
1.97486175, 0.98741104

(1.97486175, 0.98741104)